In [53]:
import scipy.sparse as sp

from src.utils import train_model, write_submission
from libs.Recommenders.Similarity.Compute_Similarity import Compute_Similarity
from src.recommender_model import RecommenderModel

# Collaborative Filtering
This notebook provies an implementation of a Collaborative Filtering model.

## User Based CF
The user-based method computes the similarity among user interactions and recommends items based on similar users.

In [54]:
class UserBasedCF(RecommenderModel):
	def __init__(self, top_k: int = 300, shrink: int = 500):
		super(UserBasedCF, self).__init__()
		self.similarity_matrix: sp.csr_matrix | None = None
		self.top_k: int = top_k
		self.shrink: int = shrink

	def fit(self, urm: sp.csr_matrix, icm: sp.csr_matrix) -> None:
		self.urm = urm

		self.similarity_matrix = Compute_Similarity(self.urm.T, topK=min(self.top_k, self.urm.shape[0]), shrink=self.shrink).compute_similarity()

		self.urm_pred = self.similarity_matrix @ self.urm

In [55]:
ubcf = train_model(UserBasedCF())

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 35736 (100.0%), 282.56 column/sec. Elapsed time 2.11 min
MAP@10 evaluation of the UserBasedCF model: 0.05459


In [60]:
ubcf_submission = train_model(UserBasedCF(), test_size=0)
write_submission(ubcf_submission, "ubcf_submission.csv")

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 35736 (100.0%), 294.99 column/sec. Elapsed time 2.02 min


## Item Based CF
The item-based method computes the similarity among the items and recommends them based on which one the user interacted with.

In [57]:
class ItemBasedCF(RecommenderModel):
	def __init__(self, top_k: int = 300, shrink: int = 500):
		super(ItemBasedCF, self).__init__()
		self.similarity_matrix: sp.csr_matrix | None = None
		self.top_k: int = top_k
		self.shrink: int = shrink

	def fit(self, urm: sp.csr_matrix, icm: sp.csr_matrix) -> None:
		self.urm = urm

		self.similarity_matrix = Compute_Similarity(self.urm, topK=min(self.top_k, self.urm.shape[1]), shrink=self.shrink).compute_similarity()

		self.urm_pred = self.urm @ self.similarity_matrix

In [58]:
ibcf = train_model(ItemBasedCF())

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 38121 (100.0%), 542.09 column/sec. Elapsed time 1.17 min
MAP@10 evaluation of the ItemBasedCF model: 0.04953


In [59]:
ibcf_submission = train_model(ItemBasedCF(), test_size=0)
write_submission(ibcf_submission, "ibcf_submission.csv")

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 38121 (100.0%), 362.89 column/sec. Elapsed time 1.75 min
